In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ======================== 回测配置（适配批量选股结果） ========================
CONFIG = {
    "selection_result_path": r'./all_selection_results_aligned.csv',  # 批量选股汇总结果
    "market_data_path": r'D:\workspace\xiaoyao\data\widetable.parquet',  # 原始行情数据
    "backtest_result_path": r'./backtest_result_aligned.csv',  # 回测明细保存路径
    "log_path": r'./backtest_log_aligned.txt'  # 回测日志
}

# ======================== 工具函数（适配字段调整） ========================
def init_backtest():
    with open(CONFIG["log_path"], 'w', encoding='utf-8') as f:
        f.write(f"【适配批量数据回测启动】{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_msg(f"✅ 初始化完成，结果路径：{CONFIG['backtest_result_path']}")

def log_msg(msg):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{current_time}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def validate_data(selection_df, market_df):
    # 验证选股结果字段（核心调整：日期字段为'trade_date'）
    sel_req = ['stock_code', 'trade_date']  # 批量结果中是'trade_date'，非'selection_date'
    if not all(col in selection_df.columns for col in sel_req):
        missing = [col for col in sel_req if col not in selection_df.columns]
        raise ValueError(f"❌ 选股结果缺字段：{missing}（当前批量结果日期字段为'trade_date'）")
    # 验证行情数据字段（保持不变）
    market_req = ['stock_code', 'date', 'open', 'close']
    if not all(col in market_df.columns for col in market_req):
        missing = [col for col in market_req if col not in market_df.columns]
        raise ValueError(f"❌ 行情数据缺字段：{missing}")
    log_msg("✅ 数据字段验证通过（适配批量选股结构）")
    return selection_df, market_df

# ======================== 核心：向量化匹配价格（适配日期字段） ========================
def match_trade_prices(selection_df, market_df):
    log_msg("开始匹配T+1/T+5价格（适配批量数据结构）...")
    
    # 1. 数据预处理（统一日期格式）
    selection_df['trade_date'] = pd.to_datetime(selection_df['trade_date']).dt.date  # 批量结果日期字段
    market_df['date'] = pd.to_datetime(market_df['date']).dt.date
    
    # 2. 行情数据加“交易日序号”（每个股票内按日期递增，0开始）
    market_df = market_df.sort_values(by=['stock_code', 'date']).reset_index(drop=True)
    market_df['trade_seq'] = market_df.groupby('stock_code').cumcount()  # 按股票分组累计序号
    log_msg(f"✅ 行情数据预处理完成：共{len(market_df)}条记录，{market_df['stock_code'].nunique()}只股票")
    
    # 3. 关联选股结果与行情数据，拿到T日序号（核心调整：用'trade_date'关联）
    market_t = market_df.rename(columns={'date': 'trade_date', 'trade_seq': 't_seq'})  # 日期字段对齐
    selection_with_t = pd.merge(
        selection_df,
        market_t[['stock_code', 'trade_date', 't_seq']],  # 用'trade_date'关联
        on=['stock_code', 'trade_date'],
        how='left'
    )
    # 过滤T日无行情的记录
    selection_with_t = selection_with_t.dropna(subset=['t_seq'])
    selection_with_t['t_seq'] = selection_with_t['t_seq'].astype(int)
    log_msg(f"✅ 关联T日序号完成：有效选股记录{len(selection_with_t)}/{len(selection_df)}条")
    
    # 4. 计算T+1和T+5序号，关联价格（逻辑不变，字段已对齐）
    selection_with_t['t1_seq'] = selection_with_t['t_seq'] + 1  # T+1序号（买入日）
    selection_with_t['t5_seq'] = selection_with_t['t_seq'] + 5  # T+5序号（卖出日）
    
    # 关联T+1开盘价（买入价）
    buy_price = market_df[['stock_code', 'trade_seq', 'open']].rename(
        columns={'trade_seq': 't1_seq', 'open': 'buy_price'}
    )
    selection_with_t = pd.merge(
        selection_with_t, buy_price, on=['stock_code', 't1_seq'], how='left'
    )
    
    # 关联T+5收盘价（卖出价）
    sell_price = market_df[['stock_code', 'trade_seq', 'close']].rename(
        columns={'trade_seq': 't5_seq', 'close': 'sell_price'}
    )
    selection_with_t = pd.merge(
        selection_with_t, sell_price, on=['stock_code', 't5_seq'], how='left'
    )
    
    # 5. 过滤无买卖价格的记录
    valid_price = selection_with_t.dropna(subset=['buy_price', 'sell_price'])
    valid_price = valid_price[(valid_price['buy_price'] > 0) & (valid_price['sell_price'] > 0)]
    log_msg(f"✅ T+1/T+5价格匹配完成：有效交易记录{len(valid_price)}/{len(selection_with_t)}条")
    
    return valid_price

# ======================== 收益计算与统计（适配批量字段） ========================
def calculate_return_and_analyze(matched_df):
    log_msg("计算收益并统计分析...")
    
    # 计算收益率（向量化）
    matched_df['return_rate'] = (matched_df['sell_price'] - matched_df['buy_price']) / matched_df['buy_price']
    
    # 核心统计（逻辑不变）
    total_valid = len(matched_df)
    avg_return = matched_df['return_rate'].mean() * 100
    positive_count = (matched_df['return_rate'] > 0).sum()
    positive_ratio = (positive_count / total_valid) * 100 if total_valid > 0 else 0
    max_return = matched_df['return_rate'].max() * 100 if total_valid > 0 else 0
    min_return = matched_df['return_rate'].min() * 100 if total_valid > 0 else 0
    median_return = matched_df['return_rate'].median() * 100 if total_valid > 0 else 0
    
    # 打印结果
    log_msg("\n" + "="*60)
    log_msg(f"📊 回测统计结果（T+1买 T+5卖）")
    log_msg("="*60)
    log_msg(f"有效交易记录：{total_valid} 条")
    log_msg(f"平均收益率：  {avg_return:.2f}%")
    log_msg(f"正收益比例：  {positive_ratio:.2f}%（{positive_count}/{total_valid}）")
    log_msg(f"最大收益率：  {max_return:.2f}%")
    log_msg(f"最小收益率：  {min_return:.2f}%")
    log_msg(f"中位数收益率：{median_return:.2f}%")
    log_msg("="*60)
    
    # 保存明细（适配批量结果字段）
    result_cols = ['stock_code', 'trade_date', 'buy_price', 'sell_price', 'return_rate', 'total_score']
    result_cols = [col for col in result_cols if col in matched_df.columns]  # 兼容可能缺失的字段
    matched_df[result_cols].to_csv(CONFIG["backtest_result_path"], index=False, encoding='utf-8-sig')
    log_msg(f"✅ 回测明细保存：{CONFIG['backtest_result_path']}")
    
    stats = {
        "avg_return": avg_return, "positive_ratio": positive_ratio,
        "max_return": max_return, "min_return": min_return, "valid_count": total_valid
    }
    return matched_df, stats

# ======================== 主流程（适配批量数据入口） ========================
def backtest_t1_buy_t5_sell():
    try:
        init_backtest()
        log_msg("="*60 + " 启动适配批量数据的回测 " + "="*60)
        
        # 1. 加载数据
        log_msg(f"\n加载选股结果：{CONFIG['selection_result_path']}")
        selection_df = pd.read_csv(CONFIG["selection_result_path"], encoding='utf-8-sig')
        
        log_msg(f"加载行情数据：{CONFIG['market_data_path']}")
        market_df = pd.read_parquet(CONFIG["market_data_path"])
        
        # 2. 验证数据（确保字段匹配）
        selection_df, market_df = validate_data(selection_df, market_df)
        
        # 3. 匹配价格（核心步骤，已适配字段）
        matched_df = match_trade_prices(selection_df, market_df)
        
        # 4. 计算收益
        if len(matched_df) == 0:
            raise ValueError("❌ 无有效交易记录，检查选股结果与行情数据的日期匹配度")
        final_df, stats = calculate_return_and_analyze(matched_df)
        
        log_msg("\n" + "="*60 + " 回测完成 " + "="*60)
        return final_df, stats
    
    except Exception as e:
        log_msg(f"❌ 回测异常：{str(e)}")
        raise

# ======================== 执行 ========================
if __name__ == "__main__":
    backtest_t1_buy_t5_sell()

[2025-10-24 11:12:16] ✅ 初始化完成，结果路径：./backtest_result_aligned.csv
[2025-10-24 11:12:16] ============================================================ 启动适配批量数据的回测 ============================================================
[2025-10-24 11:12:16] 
加载选股结果：./all_selection_results_aligned.csv
[2025-10-24 11:12:16] 加载行情数据：D:\workspace\xiaoyao\data\widetable.parquet
[2025-10-24 11:12:18] ✅ 数据字段验证通过（适配批量选股结构）
[2025-10-24 11:12:18] 开始匹配T+1/T+5价格（适配批量数据结构）...
[2025-10-24 11:12:19] ✅ 行情数据预处理完成：共998123条记录，5187只股票
[2025-10-24 11:12:19] ✅ 关联T日序号完成：有效选股记录473/473条
[2025-10-24 11:12:19] ✅ T+1/T+5价格匹配完成：有效交易记录461/473条
[2025-10-24 11:12:20] 计算收益并统计分析...
[2025-10-24 11:12:20] 
[2025-10-24 11:12:20] 📊 回测统计结果（T+1买 T+5卖）
[2025-10-24 11:12:20] ============================================================
[2025-10-24 11:12:20] 有效交易记录：461 条
[2025-10-24 11:12:20] 平均收益率：  0.34%
[2025-10-24 11:12:20] 正收益比例：  46.64%（215/461）
[2025-10-24 11:12:20] 最大收益率：  49.17%
[2025-10-24 11:12:20] 最小收益率：  -22.08%
[2025-10-24 11:12:2